In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import pickle
from tqdm.notebook import tqdm
import gc

## 1. Read multi text column files from extracted folder

In [2]:
path='../../Data/Extracted/'

train=pd.read_csv(path+'train_multi_column_text.csv')
test=pd.read_csv(path+'test_multi_column_text.csv')
labels=pd.read_csv(path+'labels.csv')
labels=pd.get_dummies(labels, prefix_sep='__')

In [5]:
from category_encoders.count import CountEncoder

In [9]:
cnt_encoder=CountEncoder()
train_cnt=cnt_encoder.fit_transform(train).div(train.shape[0])
test_cnt=cnt_encoder.transform(test).div(test.shape[0])

In [10]:
train_cnt

,Object_Description,Job_Title_Description,Sub_Object_Description,Location_Description,Function_Description,Program_Description,Fund_Description
0,0.064959,0.083807,0.763511,0.588181,0.143586,0.002399,0.145502
1,0.000402,0.039759,0.763511,0.588181,0.000363,0.000494,0.484028
2,0.019800,0.000677,0.763511,0.588181,0.143586,0.031506,0.047613
3,0.112791,0.043515,0.763511,0.588181,0.031096,0.017793,0.484028
4,0.003651,0.022573,0.763511,0.588181,0.188528,0.026689,0.484028
...,...,...,...,...,...,...,...
382622,0.041769,0.083807,0.763511,0.004764,0.007302,0.045386,0.484028
382623,0.069640,0.019949,0.763511,0.588181,0.000565,0.045386,0.484028
382624,0.064959,0.000196,0.763511,0.588181,0.143586,0.033960,0.004197
382625,0.112791,0.000112,0.763511,0.000719,0.188528,0.000251,0.484028


## 2. Label encoding with leave one out strategy with gaussian distributed noise smoothing

In [3]:
from category_encoders import leave_one_out

In [4]:
train_label_encoding_dict={}
test_label_encoding_dict={}
for label in tqdm(labels.columns):
    encoder=leave_one_out.LeaveOneOutEncoder(sigma=0.5)
    encoder.fit(train,labels[label].to_frame())
    
    train_label_encoding_dict[label]=encoder.transform(train)
    test_label_encoding_dict[label]=encoder.transform(test)

## 3. Pickle out encoded data (dictionary) in the extracted folder

In [ ]:
path='../../Data/Extracted/'

with open(path+'train_cat_feat_target_encoding.pickle','wb') as f:
    pickle.dump(train_label_encoding_dict,f)
    
with open(path+'te_cat_feat_target_encoding.pickle','wb') as f:
    pickle.dump(train_label_encoding_dict,f)